In [1]:
'''Makes pols.csv'''

# coding: utf-8
import requests
from pyquery import PyQuery as pq
from nameparser import HumanName
from slugify import slugify #for python3 requirements module is now python-slugify, but keep from slugify import slugify

import re #place before import requests
from operator import itemgetter #place before import requests
import csv #place before import requests
from time import sleep

In [2]:
HOUSEURL = "http://www.myfloridahouse.gov/Sections/Representatives/representatives.aspx"
SENATEURL = "https://www.flsenate.gov/Senators"
HOUSEPRE = "http://www.myfloridahouse.gov"
SENATEPRE = "http://www.flsenate.gov"
naptime = 0.1


In [3]:
REPLIST = []

In [4]:
SENATE = requests.get(SENATEURL).content
print(f"Processing senators at {SENATEURL}")
SENATORS = pq(SENATE)("table#Senators")
for senator in pq(SENATORS)("tr")[1:-1]:    # Skip header row and weird sorta footer row
    countiesraw = pq(senator)("tr").attr('class')
    counties = "|".join(sorted(countiesraw.split()[1:]))
    counties = counties.replace("St_", "St. ").replace("_", " ")
    personurl = SENATEPRE + pq(senator)("a").attr('href')
    alphaname = pq(senator)("a").text().replace(" , ", ", ").strip().replace("  ", " ") # Tried to fix Braynon, Oscar II
    alphaname = re.sub(r'\"\w+\"', '', alphaname).replace(" ,", " ")    # Kill off nicknames.
    alphaname = alphaname.replace(", MD, ", ", ")   # Sorry Dr. Ralph E. Massullo
    if alphaname == "Vacant":
        party = "vacant"
    else:
        if alphaname.split(", ")[1] == "Jr." or alphaname.split(", ")[1] == "Sr.":
            temp = alphaname.split(", ")
            alphaname = temp[0] + ", " + temp[2] + ", " + temp[1]
        party = pq(senator)("td")[1].text.strip()[:1]
    alphaname = re.sub(r'\s+', ' ', alphaname).replace(" , ", ", ")
    print("\t" + alphaname)
    district = pq(senator)("td")[0].text_content().strip()
    title = "Sen."
    chamber = "Senate"
    parsedname = HumanName(alphaname)
    first = parsedname.first
    last = parsedname.last
    middle = parsedname.middle
    suffix = parsedname.suffix
    if len(first) == 2: # fix for W. Travis
        first = first + " " + middle
        middle = ""
    name = first + " " + middle + " " + last + " " + suffix
    name = " ".join(name.split())       # Replace multiple spaces with one, via Jeremy Bowers and rdmurphy
    slug = slugify(title + " " + first + " " + last + " " + district)
    slug = slug.replace("NuÃ±ez", "Nunez").replace(u"Nuñez", "Nunez")
    slug = slug.lower()
    sleep(naptime)
    personhtml = requests.get(personurl).content
    biohtml = str(pq(personhtml)('div#sidebar'))
    m = re.search('(^.+?)(, FL )', biohtml, re.MULTILINE)
    city = m.group(1).strip()
    photourl = SENATEPRE + pq(biohtml)("img").attr('src')
    memberstuff = [alphaname, name, first, last, slug, title, chamber, personurl, photourl, district, party, city, counties]
    REPLIST.append(memberstuff)
print(f"Done scraping senators")

Processing senators at https://www.flsenate.gov/Senators
	Albritton, Ben
	Baxley, Dennis
	Bean, Aaron
	Benacquisto, Lizbeth
	Berman, Lori
	Book, Lauren
	Bracy, Randolph
	Bradley, Rob
	Brandes, Jeff
	Braynon, Oscar II
	Broxson, Doug
	Cruz, Janet
	Diaz, Manny Jr.
	Farmer, Gary M., Jr.
	Flores, Anitere
	Gainer, George B.
	Galvano, Bill
	Gibson, Audrey
	Gruters, Joe
	Harrell, Gayle
	Hooper, Ed
	Hutson, Travis
	Lee, Tom
	Mayfield, Debbie
	Montford, Bill
	Passidomo, Kathleen
	Perry, Keith
	Pizzo, Jason W. B.
	Powell, Bobby
	Rader, Kevin J.
	Rodriguez, Jose Javier
	Rouson, Darryl Ervin
	Simmons, David
	Simpson, Wilton
	Stargel, Kelli
	Stewart, Linda
	Taddeo, Annette
	Thurston, Perry E., Jr.
	Torres, Victor M., Jr.
	Wright, Tom A.
Done scraping senators


In [5]:
HOUSE = requests.get(HOUSEURL).content

In [6]:
print(f"Processing representatives at {HOUSEURL}")
REPS = pq(pq(HOUSE)("div.team-page")[0])
for rep in pq(REPS)("div.team-box"):
    title = "Rep."
    chamber = "House"
    alphaname = pq(rep)("h5").text().strip()
    print(f"\t{alphaname}")
    try:
        party = pq(rep)("p")[0].text.strip().split("—")[0].strip()[:1]   # Take just the first letter
    except:
        party = ""
    if "Pending District" in alphaname:
        party = "vacant"
    district = pq(pq(rep)("p")[0])("span").text().split(":")[1].strip()   # Inside the span is "District: 8". Get the 8.
    personurl = HOUSEPRE + pq(pq(rep)("a")[0]).attr('href')
    photourl = HOUSEPRE + pq(pq(rep)("img")).attr('data-src')
    slug = slugify(title + " " + first + " " + last + " " + district)
    slug = slug.replace("NuÃ±ez", "Nunez").replace(u"Nuñez", "Nunez")
    slug = slug.lower()
    countiesraw = pq(pq(rep)("p.rep-counties")).text()
    countiesraw = countiesraw.replace(" and part of", ",")
    countiesraw = countiesraw.replace(" and parts of", ",")
    countiesraw = countiesraw.replace("Part of ", "")
    countiesraw = countiesraw.replace("Parts of ", "")
    countiesraw = countiesraw.split(",")
    for i, county in enumerate(countiesraw):
        countiesraw[i] = county.strip()
    counties = "|".join(sorted(countiesraw))
    city = ""
    try:
        sleep(naptime)
        personhtml = requests.get(personurl).content
        basename = pq(personhtml)("h1")[0].text.replace("\r\n", "")
        basename = " ".join(basename.split())   # Replace random internal spaces
        basename = re.sub(r'\"\w+\"', '', basename).replace(" ,", " ")    # Kill off nicknames.
        basename = basename.replace(", MD, ", ", ")   # Sorry Dr. Ralph E. Massullo
        basename = re.sub(r'\s+', ' ', basename).replace(" , ", ", ")
        parsedname = HumanName(basename)
        first = parsedname.first
        last = parsedname.last
        middle = parsedname.middle
        suffix = parsedname.suffix
        if len(first) == 2: # fix for W. Travis
            first = first + " " + middle
            middle = ""
        name = first + " " + middle + " " + last + " " + suffix
        name = " ".join(name.split())       # Replace multiple spaces with one, via Jeremy Bowers and rdmurphy
        memberinfo = pq(personhtml)("div.mi-content")[0]
        if "city" in pq(pq(memberinfo)("span")[0]).text().lower():
            city = pq(pq(memberinfo)("span")[1]).text()
        else:
            try:
                memberinfo = pq(personhtml)("div.mi-content")[1]
                if 'city' in pq(pq(memberinfo)("span")[0]).text().lower():
                    city = pq(pq(memberinfo)("span")[1]).text()
                else:
                    city = ''
            except:
                city = ''
    except:
        pass
    memberstuff = [alphaname, name, first, last, slug, title, chamber, personurl, photourl, district, party, city, counties]
    REPLIST.append(memberstuff)
    

Processing representatives at http://www.myfloridahouse.gov/Sections/Representatives/representatives.aspx
	Alexander, Ramon
	Aloupis, Jr., Vance Arthur
	Altman, Thad
	Andrade, Robert Alexander "Alex"
	Antone, Bruce
	Ausley, Loranne
	Avila, Bryan
	Bell, Melony M.
	Beltran, Mike
	Brannan III, Robert Charles "Chuck"
	Brown, Kamia L.
	Buchanan, James
	Burton, Colleen
	Bush  III, Dr. James
	Byrd, Cord
	Caruso, Michael A. "Mike"
	Casello, Joseph A. "Joe"
	Clemons, Sr., Charles Wesley "Chuck"
	Cortes, John
	Cummings, W. Travis
	Daniels, Kimberly
	Davis, Tracie
	Diamond, Ben
	DiCeglie, Nick
	Donalds, Byron
	Drake, Brad
	Driskell, Fentrice
	DuBose, Bobby B.
	Duggan, Wyman
	Duran, Nicholas X.
	Eagle, Dane
	Eskamani, Anna  V.
	Fernández, Javier E.
	Fernandez-Barquin, Juan Alfonso
	Fetterhoff, Elizabeth Anne
	Fine, Randy
	Fischer, Jason
	Fitzenhagen, Heather
	Geller, Joseph
	Goff-Marcil, Joy
	Good, Margaret
	Gottlieb, Michael "Mike"
	Grall, Erin
	Grant, James "J.W."
	Grant, Michael
	Gregory, Tommy

In [7]:
print("Writing CSV.")
SORTEDREPS = sorted(REPLIST, key=itemgetter(0))   # Sort by last name then first, using alphaname field
with open('pols.csv', 'w', newline='') as f:
    WRITER = csv.writer(f)
    WRITER.writerow(["alphaname", "name", "first", "last", "slug", "title", "chamber", "personurl", "photourl", "district", "party", "city", "counties"])
    for row in SORTEDREPS:
        newrow = []
        for item in row:
            newrow.append(item)
        # writer.writerows(REPLIST)
        WRITER.writerow(newrow)

Writing CSV.
